## Decision Analytics Final Project
### Question: What is the best combination of tourist attractions in NYC that a travel agency can put together in order to provide the best tour experience? 
#### Team Members: Megan, Jhanvi, Hannah, Kaia

In [1]:
import numpy as np
from gurobipy import *
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
survey_table = pd.read_csv("survey_final_table.csv")
nyc = pd.read_csv("nyc attractions.csv")

In [3]:
survey_table.head()

,Unnamed: 0,Group,Proba,Nature,Museum,Family,Activity,Walking,Theatre,Landmarks
0,0,3.0,0.25,3.800000,2.000000,3.400000,4.400000,3.000000,2.400000,2.800000
1,1,2.0,0.20,1.000000,3.750000,3.000000,1.250000,2.250000,3.500000,3.000000
2,2,1.0,0.30,4.333333,3.666667,3.166667,1.833333,4.166667,3.333333,1.333333
3,3,0.0,0.25,5.000000,3.200000,1.800000,4.200000,2.600000,3.200000,1.400000


In [10]:
nyc.head()

,AttractionID,AttractionName,CostPerVisit,Nature,Museum,Landmark,FamilyFriendly,Active,Arts,Unnamed: 9,Level,Meaning
0,1,Central Park,0.0,3,2,3,3,3,2,NaN,1.0,No relevance
1,2,The National 9/11 Memorial & Museum,33.0,1,3,2,2,1,2,NaN,2.0,Some relevance
2,3,The Metropolitan Museum of Art,30.0,1,3,2,2,1,3,NaN,3.0,High relevance
3,4,Empire State Building,79.0,1,1,3,3,1,1,NaN,NaN,NaN
4,5,The High Line,0.0,3,1,3,3,3,1,NaN,NaN,NaN


In [12]:
columns_drop = ['Unnamed: 9' ,  'Level ', 'Meaning']
nyc = nyc.drop(columns=columns_drop)

In [13]:
nyc.head()

,AttractionID,AttractionName,CostPerVisit,Nature,Museum,Landmark,FamilyFriendly,Active,Arts
0,1,Central Park,0.0,3,2,3,3,3,2
1,2,The National 9/11 Memorial & Museum,33.0,1,3,2,2,1,2
2,3,The Metropolitan Museum of Art,30.0,1,3,2,2,1,3
3,4,Empire State Building,79.0,1,1,3,3,1,1
4,5,The High Line,0.0,3,1,3,3,3,1


In [14]:
nyc.columns

Index(['AttractionID', 'AttractionName', 'CostPerVisit', 'Nature', 'Museum ',
       'Landmark', 'FamilyFriendly', 'Active', 'Arts'],
      dtype='object')

In [ ]:
nyc['avg'] = nyc[['Nature', 'Museum ',
       'Landmark', 'FamilyFriendly', 'Active', 'Arts']].mean(axis=1)

In [ ]:
nyc.head()

,AttractionID,AttractionName,CostPerVisit,Nature,Museum,Landmark,FamilyFriendly,Active,Arts,avg
0,1,Central Park,0.0,3,2,3,3,3,2,2.666667
1,2,The National 9/11 Memorial & Museum,33.0,1,3,2,2,1,2,1.833333
2,3,The Metropolitan Museum of Art,30.0,1,3,2,2,1,3,2.000000
3,4,Empire State Building,79.0,1,1,3,3,1,1,1.666667
4,5,The High Line,0.0,3,1,3,3,3,1,2.333333


In [17]:
survey_table.head()

,Unnamed: 0,Group,Proba,Nature,Museum,Family,Activity,Walking,Theatre,Landmarks
0,0,3.0,0.25,3.800000,2.000000,3.400000,4.400000,3.000000,2.400000,2.800000
1,1,2.0,0.20,1.000000,3.750000,3.000000,1.250000,2.250000,3.500000,3.000000
2,2,1.0,0.30,4.333333,3.666667,3.166667,1.833333,4.166667,3.333333,1.333333
3,3,0.0,0.25,5.000000,3.200000,1.800000,4.200000,2.600000,3.200000,1.400000


In [15]:
attractions1 = nyc.to_dict()

#### Calculate the dot product of survey table responses and attraction relevance score so that we can get the happiness score of each attraction for each group

In [26]:
survey_table_short = survey_table.drop(labels=['Unnamed: 0', 'Group', 'Proba', 'Walking'], axis=1)
survey_table_short

,Nature,Museum,Family,Activity,Theatre,Landmarks
0,3.800000,2.000000,3.400000,4.400000,2.400000,2.800000
1,1.000000,3.750000,3.000000,1.250000,3.500000,3.000000
2,4.333333,3.666667,3.166667,1.833333,3.333333,1.333333
3,5.000000,3.200000,1.800000,4.200000,3.200000,1.400000


In [27]:
nyc_short = nyc[['Nature', 'Museum ', 'FamilyFriendly', 'Active', 'Arts', 'Landmark']]
nyc_short.head()

,Nature,Museum,FamilyFriendly,Active,Arts,Landmark
0,3,2,3,3,2,3
1,1,3,2,1,2,2
2,1,3,2,1,3,2
3,1,1,3,1,1,3
4,3,1,3,3,1,3


In [35]:
attraction_user_product = np.dot(nyc_short, survey_table_short.T)

attraction_user_product

array([[52.        , 39.25      , 46.        , 50.        ],
       [31.4       , 32.5       , 32.83333333, 31.6       ],
       [33.8       , 36.        , 36.16666667, 34.8       ],
       [31.2       , 27.5       , 26.66666667, 25.2       ],
       [47.6       , 32.        , 39.        , 43.6       ],
       [31.2       , 27.5       , 26.66666667, 25.2       ],
       [35.        , 28.5       , 31.        , 30.2       ],
       [27.8       , 24.5       , 23.5       , 23.4       ],
       [27.        , 25.5       , 27.5       , 27.        ],
       [30.2       , 28.        , 26.83333333, 26.6       ],
       [37.4       , 32.        , 34.33333333, 33.4       ],
       [41.        , 40.        , 43.66666667, 41.6       ],
       [32.2       , 25.5       , 29.66666667, 28.8       ],
       [31.2       , 27.5       , 26.66666667, 25.2       ],
       [27.8       , 24.5       , 23.5       , 23.4       ],
       [32.2       , 31.75      , 30.5       , 29.8       ],
       [31.2       , 27.

In [ ]:
import numpy as np
from gurobipy import *
import pandas as pd

# Load data
survey_table = pd.read_csv("survey_final_table.csv")
nyc = pd.read_csv("nyc attractions.csv")

# Clean and preprocess data
columns_drop = ['Unnamed: 9', 'Level', 'Meaning']
nyc = nyc.drop(columns=[col for col in columns_drop if col in nyc.columns])

# Add average column
nyc['avg'] = nyc[['Nature', 'Museum ', 'Landmark', 'FamilyFriendly', 'Active', 'Arts']].mean(axis=1)

# Convert to dictionary for optimization
attractions1 = nyc.set_index('AttractionID').to_dict('index')

# Parameters
budget = 1000  # Total budget constraint
max_duration = 8  # Maximum number of hours available for the tour
demographic_weights = [0.25, 0.20, 0.30, 0.25]  # Example weights for demographic groups

# Initialize Model
model = Model("NYC_Tour_Optimization")

# Decision Variables
A = model.addVars(len(attractions1), vtype=GRB.BINARY, name="A")  # Binary decision for each attraction

# Objective Function
happiness_scores = [
    demographic_weights[j] * attraction_user_product[i][j]
    for i in range(24) # Loop through every attraction
    for j in range(len(demographic_weights)) # Loop through every group
]

# Align scores with the attraction index
#adjusted_scores = [
#    sum(demographic_weights[j] * attractions1[i + 1]["avg"] for j in range(len(demographic_weights)))
#    for i in range(len(attractions1))
#]

#model.setObjective(quicksum(A[i] * adjusted_scores[i] for i in range(len(attractions1))), GRB.MAXIMIZE)
#model.setObjective(quicksum(A[i] * happiness_scores[i] for i in range(24)), GRB.MAXIMIZE)
model.setObjective(quicksum(demographic_weights[j] * A[i] * attraction_user_product[i][j] for i in range(24) for j in range(3)), GRB.MAXIMIZE)

### Constraints ###

# Budget Constraint
model.addConstr(
    quicksum(A[i] * attractions1[i + 1]["CostPerVisit"] for i in range(len(attractions1))) <= budget, "Budget"
)

# Total Duration Constraint (tour cannot exceed available time)
attraction_durations = [1.5 for _ in range(len(attractions1))]  # Example durations for each attraction in hours
model.addConstr(
    quicksum(A[i] * attraction_durations[i] for i in range(len(attractions1))) <= max_duration, "Duration"
)

# Constraints for specific categories
categories = ["Nature", "Museum ", "Landmark", "FamilyFriendly", "Active", "Arts"]
category_min_ratios = [2 / 23, 1 / 23, 5 / 23, 0.5, 1 / 23, 3 / 23]  # Example minimum ratios

for category, min_ratio in zip(categories, category_min_ratios):
    model.addConstr(
        quicksum(A[i] * attractions1[i + 1][category] for i in range(len(attractions1))) 
        >= min_ratio * quicksum(A[i] for i in range(len(attractions1))), f"{category}_Constraint"
    )

# Pairwise Constraints (e.g., cannot select both Central Park and 9/11 Memorial)
model.addConstr(A[0] + A[1] <= 1, "Pairwise_Constraint")

# Optimize
model.optimize()

# Print Results
if model.status == GRB.OPTIMAL:
    selected_attractions = [i + 1 for i in range(len(attractions1)) if A[i].X > 0.5]
    print("Optimal Attractions:", selected_attractions)
    print("Maximum Happiness:", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))



CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 9 rows, 24 columns and 180 nonzeros
Model fingerprint: 0xaf3ca723
Variable types: 0 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [8e-01, 1e+02]
  Objective range  [2e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 133.7500000
Presolve removed 9 rows and 24 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 152.45 133.75 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.524500000000e+02, best bound 1.524500000000e+02, gap 0.0000%
Optimal Attractions: [1, 5, 12, 18, 24]
Maximum Happiness: 152.45
